<a href="https://colab.research.google.com/github/ganeshmylavarapu-qp/OQ_PROJECT/blob/main/Log_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import re
import numpy as np
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
import math
import random
import sys
import matplotlib.pyplot as plt
from keras.utils import plot_model

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
data_path = "/content/gdrive/MyDrive/Ganesh_Latex_test.log"
raw_text = open(data_path).read().lower()

print(raw_text)

# generate list of unique characters, but only include words and some punctuation marks
pattern = re.compile('[a-z]+|\!|\n|\.|,|;')
all_words = re.findall(pattern, raw_text)

unique_words = sorted(set(all_words))

word_to_int = dict((c, i) for i, c in enumerate(unique_words))

# print(unique_words)
# print(word_to_int)

# later used to make outputs more readable by converting ints back to characters
int_to_word = dict((i, c) for i, c in enumerate(unique_words))

total_num_words = len(all_words)
len_vocab = len(unique_words)

print("Total number of words:\t" + str(total_num_words))
print("Length of vocabulary:\t" + str(len_vocab))

this is pdftex, version 3.14159265-2.6-1.40.18 (tex live 2017) (preloaded format=pdflatex 2017.5.23)  9 sep 2017 16:37
entering extended mode
 restricted \write18 enabled.
 file:line:error style messages enabled.
 %&-line parsing enabled.
**ganesh_latex_test.tex
(./ganesh_latex_test.tex
latex2e <2017-04-15>
babel <3.10> and hyphenation patterns for 84 language(s) loaded.
(/usr/local/texlive/2017/texmf-dist/tex/latex/base/article.cls
document class: article 2014/09/29 v1.4h standard latex document class
(/usr/local/texlive/2017/texmf-dist/tex/latex/base/size10.clo
file: size10.clo 2014/09/29 v1.4h standard latex file (size option)
)
\c@part=\count79
\c@section=\count80
\c@subsection=\count81
\c@subsubsection=\count82
\c@paragraph=\count83
\c@subparagraph=\count84
\c@figure=\count85
\c@table=\count86
\abovecaptionskip=\skip41
\belowcaptionskip=\skip42
\bibindent=\dimen102
)
(/usr/local/texlive/2017/texmf-dist/tex/latex/mathtools/mathtools.sty
package: mathtools 2015/11/12 v1.18 mathemati

In [6]:
sequence_length = 25 # max number of words to consider at a time.
                    # this means that each trainig set (training pattern) will be comprised of 20 time steps
step_window = 3

# set up x and y
# convert the words into integers
x_data = [] # list of lists
y_data = []

for i in range(0, total_num_words - sequence_length, step_window):
    
    # extract the first n words (length sequence_length): our "x"
    sequence_in = all_words[i : i+sequence_length]
    
    # extract last word for this window: our "y" (target)
    word_out = all_words[i+sequence_length]
    
    # print('\nx: ' + str(sequence_in) + '\n' + 'y: ' + word_out)
    # print()
        
    # store corresponding integer for each character in the input sequence
    x_data.append(sequence_in)
    y_data.append(word_out)

num_train_patters = len(x_data)
print('Total patterns:\t' + str(num_train_patters))

Total patterns:	576


In [7]:

x = np.zeros((num_train_patters, sequence_length, len_vocab))
y = np.zeros((num_train_patters, len_vocab))

# encode all data into one-hot vectors
for i, sentence in enumerate(x_data):
    for t, word in enumerate(sentence):
        x[i, t, word_to_int[word]] = 1
    y[i, word_to_int[y_data[i]]] = 1

In [11]:
learning_rate = 0.01
optimizer = RMSprop(lr=learning_rate)
num_memory_units = 256

model = Sequential()

# model.add(LSTM(num_memory_units, return_sequences=True, input_shape=(sequence_length, len_vocab)))
model.add(LSTM(num_memory_units, input_shape=(sequence_length, len_vocab)))
model.add(Dropout(0.2))
# model.add(LSTM(num_memory_units))
# model.add(Dropout(0.2))
model.add(Dense(len_vocab))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [12]:
def add_temperature(predictions, temperature=1.0):

    predictions = np.asarray(predictions).astype('float64')
    predictions = np.log(predictions) / temperature
    exp_predictions = np.exp(predictions)
    predictions = exp_predictions / np.sum(exp_predictions)
    
    probabilities = np.random.multinomial(1, predictions, 1)
    return np.argmax(probabilities)

In [15]:
num_iterations = 50
batch_size = 128
words_to_generate = 300

prev_loss = math.inf
loss_history = []
accuracy_history = []

val_loss_history = []
val_accuracy_history = []

# train the model, output generated text after each iteration
for i in range(num_iterations):
    
    print('\n' + '-'*10 + ' epoch ' + str(i+1) + '/' + str(num_iterations) + ' ' + '-'*10)
        
    history = model.fit(x, y, batch_size=batch_size, epochs=1)
    
    curr_loss = history.history['loss'][0]
    loss_history.append(curr_loss)
    
    # save weights if loss improves
    if (curr_loss < prev_loss):
        print("Loss improved from " + str(prev_loss) + " to " + str(curr_loss) + ". Saving weights.")
        model.save_weights('weights_epoch-{}_loss-{}.hdf5'.format(i, curr_loss))
        prev_loss = curr_loss
    
    start_index = random.randint(0, total_num_words - sequence_length - 1)
    # start_index = 0

    seed_sentence = all_words[start_index : start_index + sequence_length]

    print('\n-> seed: "' + ' '.join(seed_sentence) + '" ...\n')

    for i in range(words_to_generate):
        
        x_input = np.zeros((1, sequence_length, len_vocab))
        for t, word in enumerate(seed_sentence):
            x_input[0, t, word_to_int[word]] = 1.

        predictions = model.predict(x_input, verbose=0)[0]
        
        if i == num_iterations-1:
            final_predicted = predictions
        
        # predicted_word_index = add_temperature(predictions, 0.5)
        predicted_word_index = np.argmax(predictions)
        predicted_word = int_to_word[predicted_word_index]

        seed_sentence = seed_sentence[1:] + list([predicted_word])

        if re.match('[a-z]', predicted_word):
            sys.stdout.write(" " + predicted_word)
        else:
            sys.stdout.write(predicted_word)
    
        sys.stdout.flush()
        
    print()


---------- epoch 1/50 ----------
5/5 [==============================] - 1s 205ms/step - loss: 0.3677
Loss improved from inf to 0.3676736652851105. Saving weights.

-> seed: "
 andhelp toks 
 eqnshift dimen 
 alignsep dimen 
 tagshift dimen 
 tagwidth dimen 
 totwidth dimen 
 lineht dimen 
 envbody toks 
" ...


 usr local texlive texmf dist tex generic oberdiek ifpdf. sty
 package kvoptions v. a programming kkt features
 package pdftexcmds v driver not detected ifluatex

 l mt above intertext sep dimen
 ganesh usr local texlive texmf var tex latex oberdiek infwarerr. sty
 package amsmath v. d standard kkt dpc class
 usr local texlive texmf dist tex latex oberdiek grfext. sty
 package trig v. a name parser features.

 paragraph local texlive texmf dist tex latex base grfext. sty
 package kvoptions v. key value ifluatex dpc.
 usr local texlive texmf dist tex latex amsmath amsgen. sty
 package trig v. define name parser dpc

 see local texlive texmf dist tex latex amsmath amsgen. sty
 pa